In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=01c22ba08b5864b35a2e504f5ad2ab84f5d1ac759c4a8455c29a377f7dd06898
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


Есть мастер (закрепляется либо на моем компьютере либо на кластере). Подается программа-драйвер, менеджер кластера проверит рабочие места с доступным пространством и нанимает рабочих.

Информация о мощности кодируется в SparkContext - подключение к кластеру.

* Менеджер
* Драйвер
* Worker
* Исполнитель

Действие (ввод-вывод) и преобразование. count() и то и то

In [ ]:
# какие самые популярные слова используются в английском языке («Гордость и предубеждение» Джейн Остин)
# начнем с настройки PySpark shell
# Python предоставляет REPL для интерактивной разработки. Поскольку PySpark является библиотекой Python, она также использует ту же среду

from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("Analyzing the vocabulary of Pride and Prejudice.") .getOrCreate())

In [ ]:
# data frame (структура данных) бытсрее чем RDD (низкоуровневый)
spark.sparkContext # не запускать

<SparkContext master=local[*] appName=Analyzing the vocabulary of Pride and Prejudice.>

In [ ]:
# начинаем с уровней ведения журнала (levels of logging) (конспект)
spark.sparkContext.setLogLevel("WARN")

In [ ]:
# структура RDD (строки) и data frame (столбцы)
# имя модуля для обработки данных pyspark.sql

spark.read # подгрузка через объект DataFrameReader
dir(spark.read)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_df',
 '_jreader',
 '_set_opts',
 '_spark',
 'csv',
 'format',
 'jdbc',
 'json',
 'load',
 'option',
 'options',
 'orc',
 'parquet',
 'schema',
 'table',
 'text']

In [ ]:
book = spark.read.text("1342-0.txt")
book # вывод схема столбцы с типом (у нас строка с символом разделения строки)

DataFrame[value: string]

In [ ]:
book.printSchema()

root
 |-- value: string (nullable = true)



In [ ]:
print(book.dtypes) # для фильтрации схемы

[('value', 'string')]


In [ ]:
book.show()

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|                    |
|Title: Pride and ...|
|                    |
| Author: Jane Austen|
|                    |
|Posting Date: Aug...|
|Release Date: Jun...|
|Last Updated: Mar...|
|                    |
|   Language: English|
|                    |
|Character set enc...|
|                    |
+--------------------+
only showing top 20 rows



In [ ]:
book.show(10, truncate=50, vertical = False)#строки, усечение до 20 или определенное кол-во, vertical

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|The Project Gutenberg EBook of Pride and Prejud...|
|                                                  |
|This eBook is for the use of anyone anywhere at...|
|almost no restrictions whatsoever.  You may cop...|
|re-use it under the terms of the Project Gutenb...|
|    with this eBook or online at www.gutenberg.org|
|                                                  |
|                                                  |
|                        Title: Pride and Prejudice|
|                                                  |
+--------------------------------------------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import split
lines = book.select(split(book.value, " ").alias("line"))
lines.show(5) # строки в список и с пустотой

+--------------------+
|                line|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows



In [ ]:
book.select(book.value)

DataFrame[value: string]

In [ ]:
from pyspark.sql.functions import col
book.select(book.value)
book.select(book["value"])
book.select(col("value")) # из модуля
book.select("value")

DataFrame[value: string]

In [ ]:
from pyspark.sql.functions import col, split
lines = book.select(split(col("value"), " ")) # есть третий параметр
lines.printSchema()

root
 |-- split(value,  , -1): array (nullable = true)
 |    |-- element: string (containsNull = false)



In [ ]:
lines.show(5)

+--------------------+
| split(value,  , -1)|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows



In [ ]:
#переименование
book.select(split(col("value"), " ")).printSchema()

root
 |-- split(value,  , -1): array (nullable = true)
 |    |-- element: string (containsNull = false)



In [ ]:
book.select(split(col("value"), " ").alias("line")).printSchema()

root
 |-- line: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [ ]:
# когда используется метод какие столбцы должны отображаться (как select)
lines = lines.withColumnRenamed("split(value,  , -1)", "line")
lines.printSchema()

root
 |-- line: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [ ]:
#изменение формы данных
#список в строке преобразуем в отдельное слово в столбце - explode
from pyspark.sql.functions import explode, col
words = lines.select(explode(col("line")).alias("word"))
words.show(15)

+----------+
|      word|
+----------+
|       The|
|   Project|
| Gutenberg|
|     EBook|
|        of|
|     Pride|
|       and|
|Prejudice,|
|        by|
|      Jane|
|    Austen|
|          |
|      This|
|     eBook|
|        is|
+----------+
only showing top 15 rows



In [ ]:
# изменение регистра и удаление знаков препинания
from pyspark.sql.functions import lower
words_lower = words.select(lower(col("word")).alias("word_lower"))
words_lower.show(5)

+----------+
|word_lower|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
+----------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import regexp_extract # регулярные выражение
words_clean = words_lower.select(regexp_extract(col("word_lower"), "[a-z]+", 0).alias("word"))
words_clean.show(5)

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
+---------+
only showing top 5 rows



In [ ]:
# фильрация строк либо filter либо where
words_nonull = words_clean.filter(col("word") != "")

**Глава 3. Submitting and scaling первой программы PySpark**

In [ ]:
# объект GroupedData, сгруппированный кадр в этом объекте
results = words_nonull.groupby(col("word")).count()
print(results) # DataFrame[word: string, count: bigint]
results.show(5)

DataFrame[word: string, count: bigint]
+------+-----+
|  word|count|
+------+-----+
|online|    4|
|  some|  209|
| still|   72|
|   few|   72|
|  hope|  122|
+------+-----+
only showing top 5 rows



In [ ]:
results.orderBy("count", ascending=False).show(5)
results.orderBy(col("count").desc()).show(5)

+----+-----+
|word|count|
+----+-----+
| the| 4496|
|  to| 4235|
|  of| 3719|
| and| 3602|
| her| 2223|
+----+-----+
only showing top 5 rows

+----+-----+
|word|count|
+----+-----+
| the| 4496|
|  to| 4235|
|  of| 3719|
| and| 3602|
| her| 2223|
+----+-----+
only showing top 5 rows



In [ ]:
# записывание данных в csv из data frame
results.write.csv("./data/simple_count.csv")
# создался каталог с тем же именем и поместил в него 201 файл (200 CSV + 1 файл _SUCCESS)
# файл _SUCCESS означает что операция прошла успешно, и 200 файлов

In [ ]:
results.coalesce(1).write.csv("./data/simple_count_single_partition.csv") # один файл

In [ ]:
# упрощение кода через соглашения об импорте
# Before
from pyspark.sql.functions import col, explode, lower, regexp_extract, split
# After
import pyspark.sql.functions as F

In [ ]:
# упрощение
# select(), where(), groupBy() принимают структуру в качестве параметра data frame
# или GroupedData для count() и возвращают структуру
# принимают структуру и возвращают структуру (dataframe или GroupedData)

results.orderBy("word").count()
#для такого нужно присовить переменной т.к нет выполнения на месте
import pyspark.sql.functions as F
results = (spark.read.text("./data/gutenberg_books/1342-0.txt")
                .select(F.split(F.col("value"), " ").alias("line"))
                .select(F.explode(F.col("line")).alias("word"))
                .select(F.lower(F.col("word")).alias("word"))
                .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
                .where(F.col("word") != "")
                .groupby("word").count())
#два действия две переменные, одно действие можно и без переменной

In [ ]:
# batch mode, достаточно определить свой SparkSession

results = spark.read.text('./data/gutenberg_books/*.txt')
#берем все файлы каталога

**Глава 4. Analyzing tabular data
with pyspark.sql**

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.getOrCreate()

#структура PySpark это data frame или объект Column

my_grocery_list = [["Banana", 2, 1.74], ["Apple", 4, 2.04], ["Carrot", 1, 1.09], ["Cake", 1, 10.99],]
df_grocery_list = spark.createDataFrame(my_grocery_list, ["Item", "Quantity", "Price"])
df_grocery_list.printSchema()
# на вход createDataFrame: список списков, data frame pandas или RDD + схема

root
 |-- Item: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Price: double (nullable = true)



In [ ]:
.toPandas() # для перевода в DataFrame и графиков

67 - 90

In [ ]:
!git clone https://github.com/jonesberg/DataAnalysisWithPythonAndPySpark-Data

Cloning into 'DataAnalysisWithPythonAndPySpark-Data'...
remote: Enumerating objects: 71, done.
remote: Total 71 (delta 0), reused 0 (delta 0), pack-reused 71
Receiving objects: 100% (71/71), 828.81 MiB | 24.76 MiB/s, done.
Updating files: 100% (59/59), done.


In [ ]:
# подробнее про SparkReader, начинаем с csv (где есть разделитель строки и полей)
#

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("Analyzing") .getOrCreate())

In [ ]:
logs = spark.read.csv('/content/DataAnalysisWithPythonAndPySpark-Data/broadcast_logs/BroadcastLogs_2018_Q3_M8_sample.CSV',
                       sep="|", header=True,  inferSchema=True, timestampFormat="yyyy-MM-dd")
# inferSchema=True есть True то сам определяет тип данных

In [ ]:
logs.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable 

In [ ]:
logs.select("BroadcastLogID", "LogServiceID", "LogDate").show(5, False)

+--------------+------------+----------+
|BroadcastLogID|LogServiceID|LogDate   |
+--------------+------------+----------+
|1196192316    |3157        |2018-08-01|
|1196192317    |3157        |2018-08-01|
|1196192318    |3157        |2018-08-01|
|1196192319    |3157        |2018-08-01|
|1196192320    |3157        |2018-08-01|
+--------------+------------+----------+
only showing top 5 rows



In [ ]:
logs.select("BroadCastLogID", "LogServiceID", "LogDate")
logs.select(*["BroadCastLogID", "LogServiceID", "LogDate"])
logs.select(F.col("BroadCastLogID"), F.col("LogServiceID"), F.col("LogDate"))
logs.select(*[F.col("BroadCastLogID"), F.col("LogServiceID"), F.col("LogDate")])

In [ ]:
import numpy as np
column_split = np.array_split(np.array(logs.columns), len(logs.columns) // 3)
print(column_split)

for x in column_split:
 logs.select(*x).show(5, False)

[array(['BroadcastLogID', 'LogServiceID', 'LogDate'], dtype='<U22'), array(['SequenceNO', 'AudienceTargetAgeID', 'AudienceTargetEthnicID'],
      dtype='<U22'), array(['CategoryID', 'ClosedCaptionID', 'CountryOfOriginID'], dtype='<U22'), array(['DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID'],
      dtype='<U22'), array(['ProgramClassID', 'FilmClassificationID', 'ExhibitionID'],
      dtype='<U22'), array(['Duration', 'EndTime', 'LogEntryDate'], dtype='<U22'), array(['ProductionNO', 'ProgramTitle', 'StartTime'], dtype='<U22'), array(['Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID'],
      dtype='<U22'), array(['BroadcastOriginPointID', 'CompositionID', 'Producer1'],
      dtype='<U22'), array(['Producer2', 'Language1', 'Language2'], dtype='<U22')]
+--------------+------------+----------+
|BroadcastLogID|LogServiceID|LogDate   |
+--------------+------------+----------+
|1196192316    |3157        |2018-08-01|
|1196192317    |3157        |2018-08-01|
|1196192318    |

In [ ]:
logs = logs.drop("BroadcastLogID", "SequenceNO")

print("BroadcastLogID" in logs.columns)
print("SequenceNo" in logs.columns)

False
False


In [ ]:
logs = logs.select(*[x for x in logs.columns if x not in ["BroadcastLogID", "SequenceNO"]])

In [ ]:
logs.select(F.col("Duration")).show(5)

+----------------+
|        Duration|
+----------------+
|02:00:00.0000000|
|00:00:30.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
+----------------+
only showing top 5 rows



In [ ]:
print(logs.select(F.col("Duration")).dtypes)

[('Duration', 'string')]


In [ ]:
logs.select(F.col("Duration"),
            F.col("Duration").substr(1, 2).cast("int").alias("dur_hours"),
            F.col("Duration").substr(4, 2).cast("int").alias("dur_minutes"),
            F.col("Duration").substr(7, 2).cast("int").alias("dur_seconds")).distinct().show(5)
#substr(начало с 1 (а не 0) и длина), cast ставим тим

+----------------+---------+-----------+-----------+
|        Duration|dur_hours|dur_minutes|dur_seconds|
+----------------+---------+-----------+-----------+
|00:04:52.0000000|        0|          4|         52|
|00:10:06.0000000|        0|         10|          6|
|00:26:41.0000000|        0|         26|         41|
|00:05:29.0000000|        0|          5|         29|
|00:08:18.0000000|        0|          8|         18|
+----------------+---------+-----------+-----------+
only showing top 5 rows



In [ ]:
logs.select(F.col("Duration"),
 (F.col("Duration").substr(1, 2).cast("int") * 60 * 60 +
  F.col("Duration").substr(4, 2).cast("int") * 60 +
  F.col("Duration").substr(7, 2).cast("int")).alias("Duration_seconds")).distinct().show(5)

+----------------+----------------+
|        Duration|Duration_seconds|
+----------------+----------------+
|01:59:30.0000000|            7170|
|00:31:00.0000000|            1860|
|00:28:08.0000000|            1688|
|00:10:30.0000000|             630|
|00:32:00.0000000|            1920|
+----------------+----------------+
only showing top 5 rows



In [ ]:
logs = logs.withColumn("Duration_seconds",
 (F.col("Duration").substr(1, 2).cast("int") * 60 * 60
 + F.col("Duration").substr(4, 2).cast("int") * 60
 + F.col("Duration").substr(7, 2).cast("int")))
logs.printSchema()

root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttentionID: inte

In [ ]:
82-90 90-108
# переименовать столбец
logs = logs.withColumnRenamed("Duration_seconds", "duration_seconds")
logs.printSchema()

# переименовать махом все столбцы без цикла с toDF
logs.toDF(*[x.lower() for x in logs.columns]).printSchema()

#изменение порядка через select
logs.select(sorted(logs.columns)).printSchema()

In [ ]:
# описание через describe() - все столбцы и число и строки, и summary() для числовых столбцов
for i in logs.columns:
  logs.describe(i).show()

# summary() добавляет 25-50 и 75 процентили
for i in logs.columns:
  logs.select(i).summary().show()

for i in logs.columns:
  logs.select(i).summary("min", "10%", "90%", "max").show()

#оба метода с ненулевыми значениями но count будет считать ненулевые

**Глава 5 Data frame gymnastics:
Joining and grouping**

In [ ]:
# join many to one
DIRECTORY = "./data/broadcast_logs"
log_identifier = spark.read.csv("ReferenceTables/LogIdentifier.csv", sep="|", header=True, inferSchema=True)
log_identifier.printSchema()

log_identifier = log_identifier.where(F.col("PrimaryFG") == 1)
print(log_identifier.count())

log_identifier.show(5)

In [ ]:
logs.join(log_identifier, on=[PREDICATES], how=[METHOD])
# on: e logs["LogServiceID"] == log_identifier["LogServiceID"]
# (logs["LogServiceID"] == log_identifier["LogServiceID"]) & (logs["left_col"] < log_identifier["right_col"])
# [left["col1"] == right["colA"], left["col2"] > right["colB"], left["col3"] != right["colC"]]
# logs.join(log_identifier, on="LogServiceID", how=[METHOD])

In [ ]:
how="inner" # объединение на одном компьютере - shuffle
how="left" or how="left_outer" and right how="right" or how="right_outer"
how="left_semi" # как inner но остаются столбцы из левой таблицы (без дублирования)
how="left_anti" # записи левой что не соотвествуют правой
how="cross"

In [ ]:
logs_and_channels_verbose = logs.join(log_identifier, logs["LogServiceID"] == log_identifier["LogServiceID"])
logs_and_channels_verbose.printSchema() # выдаст два лдинаковых столбца

logs_and_channels = logs.join(log_identifier, "LogServiceID")
logs_and_channels.printSchema() # этот способ удалит повторяющиеся

In [ ]:
logs_and_channels_verbose = logs.join(log_identifier, logs["LogServiceID"] == log_identifier["LogServiceID"])
logs_and_channels.drop(log_identifier["LogServiceID"]).select("LogServiceID") # join помнит откуда пришел столбец

In [ ]:
logs_and_channels_verbose = logs.alias("left").join(
    log_identifier.alias("right"),
    logs["LogServiceID"] == log_identifier["LogServiceID"])
logs_and_channels_verbose.drop(F.col("right.LogServiceID")).select("LogServiceID")

In [ ]:
IRECTORY = "./data/broadcast_logs"
cd_category = spark.read.csv(os.path.join(DIRECTORY, "ReferenceTables/CD_Category.csv"),
 sep="|",
 header=True,
 inferSchema=True).select("CategoryID", "CategoryCD", F.col("EnglishDescription").alias("Category_Description"))
cd_program_class = spark.read.csv(os.path.join(DIRECTORY, "ReferenceTables/CD_ProgramClass.csv"),
 sep="|",
 header=True,
 inferSchema=True).select("ProgramClassID", "ProgramClassCD", F.col("EnglishDescription").alias("ProgramClass_Description"))
full_log = logs_and_channels.join(cd_category, "CategoryID", how="left").join(cd_program_class, "ProgramClassID", how="left")

Обобщение данных с помощью groupby

Могли бы использовать метод sum() напрямую, но у нас не было бы возможности псевдонима результирующего столбца, и мы бы застряли с sum(duration_seconds) для имени. Вместо этого мы используем странно названный agg().

In [ ]:
full_log
 .groupby("ProgramClassCD", "ProgramClass_Description")
 .agg(F.sum("duration_seconds").alias("duration_total"))
 .orderBy("duration_total", ascending=False).show(100, False)

In [ ]:
# упрощение
full_log.groupby("ProgramClassCD", "ProgramClass_Description").agg(
 {"duration_seconds": "sum"}
).withColumnRenamed("sum(duration_seconds)", "duration_total").
  orderBy("duration_total", ascending=False).show(100, False)

In [ ]:
F.when(F.trim(F.col("ProgramClassCD")).isin(
["COM", "PRC", "PGI", "PRO", "PSA", "MAG", "LOC", "SPO", "MER", "SOL"]),
F.col("duration_seconds")).otherwise(0) # если то иначе ноль

In [ ]:
(F.when([BOOLEAN TEST], [RESULT IF TRUE])
 .when([ANOTHER BOOLEAN TEST], [RESULT IF TRUE])
 .otherwise([DEFAULT RESULT, WILL DEFAULT TO null IF OMITTED]))

In [ ]:
answer = (full_log.groupby("LogIdentifierID")
 .agg(F.sum(F.when(F.trim(F.col("ProgramClassCD")).isin(
 ["COM", "PRC", "PGI", "PRO", "LOC", "SPO", "MER", "SOL"]),
 F.col("duration_seconds")).otherwise(0)).alias("duration_commercial"),
 F.sum("duration_seconds").alias("duration_total")).withColumn("commercial_ratio", F.col(
 "duration_commercial") / F.col("duration_total")))

answer.orderBy("commercial_ratio", ascending=False).show(1000, False)

Удаление пропусков

In [ ]:
# dropna() how = any или all
# trech - принимает целое число и отбросит записи со значением
# subset - столбцы для принятия решения
answer_no_null = answer.dropna(subset=["commercial_ratio"])
answer_no_null.orderBy("commercial_ratio", ascending=False).show(1000, False)
print(answer_no_null.count()) # 322

In [ ]:
# value - чем но совместимым
# subset - столбцы

answer_no_null = answer.fillna(0)
answer_no_null.orderBy("commercial_ratio", ascending=False).show(1000, False)

In [ ]:
answer_no_null = answer.fillna(
 {"duration_commercial": 0, "duration_total": 0, "commercial_ratio": 0})